<a href="https://colab.research.google.com/github/sachin886x/deep-learning-lab/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision tqdm


In [2]:
!unzip cats_vs_dogs.zip -d /content/cats_dogs


Streaming output truncated to the last 5000 lines.
  inflating: /content/cats_dogs/train/Dog/5489.jpg  
  inflating: /content/cats_dogs/train/Dog/549.jpg  
  inflating: /content/cats_dogs/train/Dog/5490.jpg  
  inflating: /content/cats_dogs/train/Dog/5491.jpg  
  inflating: /content/cats_dogs/train/Dog/5492.jpg  
  inflating: /content/cats_dogs/train/Dog/5493.jpg  
  inflating: /content/cats_dogs/train/Dog/5494.jpg  
  inflating: /content/cats_dogs/train/Dog/5495.jpg  
  inflating: /content/cats_dogs/train/Dog/5496.jpg  
  inflating: /content/cats_dogs/train/Dog/5497.jpg  
  inflating: /content/cats_dogs/train/Dog/5498.jpg  
  inflating: /content/cats_dogs/train/Dog/5499.jpg  
  inflating: /content/cats_dogs/train/Dog/55.jpg  
  inflating: /content/cats_dogs/train/Dog/550.jpg  
  inflating: /content/cats_dogs/train/Dog/5500.jpg  
  inflating: /content/cats_dogs/train/Dog/5501.jpg  
  inflating: /content/cats_dogs/train/Dog/5502.jpg  
  inflating: /content/cats_dogs/train/Dog/5503.jpg  

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])


In [6]:
train_data_cats = datasets.ImageFolder(
    root="/content/cats_dogs/train",
    transform=transform
)

train_loader_cats = DataLoader(
    train_data_cats,
    batch_size=32,
    shuffle=True
)


In [7]:
train_data_cifar = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

train_loader_cifar = DataLoader(
    train_data_cifar,
    batch_size=32,
    shuffle=True
)


100%|██████████| 170M/170M [00:13<00:00, 12.8MB/s]


In [8]:
class CustomCNN(nn.Module):
    def __init__(self, activation_fn):
        super(CustomCNN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            activation_fn,
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            activation_fn,
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            activation_fn,
            nn.MaxPool2d(2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256),
            activation_fn,
            nn.Dropout(0.5),
            nn.Linear(256, 10)  # 10 for CIFAR, 2 for Cats/Dogs
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [9]:
def initialize_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            if init_type == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_normal_(m.weight)
            elif init_type == "random":
                nn.init.normal_(m.weight, mean=0, std=0.01)


In [10]:
def train_model(model, loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        correct = 0
        total = 0

        for images, labels in tqdm(loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")


In [11]:
activations = {
    "relu": nn.ReLU(),
    "tanh": nn.Tanh(),
    "leaky_relu": nn.LeakyReLU()
}

optimizers = {
    "sgd": optim.SGD,
    "adam": optim.Adam,
    "rmsprop": optim.RMSprop
}

initializations = ["xavier", "kaiming", "random"]


In [12]:
model = CustomCNN(activations["relu"]).to(device)
initialize_weights(model, "kaiming")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader_cifar, optimizer, criterion)


100%|██████████| 1563/1563 [03:07<00:00,  8.35it/s]


Epoch 1, Loss: 4501.5187, Accuracy: 9.82%


100%|██████████| 1563/1563 [03:05<00:00,  8.43it/s]


Epoch 2, Loss: 3599.2461, Accuracy: 9.92%


100%|██████████| 1563/1563 [03:05<00:00,  8.41it/s]


Epoch 3, Loss: 3599.2658, Accuracy: 9.74%


100%|██████████| 1563/1563 [03:05<00:00,  8.41it/s]


Epoch 4, Loss: 3599.2445, Accuracy: 9.79%


100%|██████████| 1563/1563 [03:05<00:00,  8.44it/s]

Epoch 5, Loss: 3599.2666, Accuracy: 9.70%


In [13]:
torch.save(model.state_dict(), "best_cnn_cifar.pth")


In [14]:
resnet = models.resnet18(pretrained=True)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 207MB/s]


In [15]:
resnet.fc = nn.Linear(resnet.fc.in_features, 10)  # CIFAR-10
resnet = resnet.to(device)


In [16]:
optimizer = optim.Adam(resnet.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

train_model(resnet, train_loader_cifar, optimizer, criterion, epochs=3)


100%|██████████| 1563/1563 [03:35<00:00,  7.27it/s]


Epoch 1, Loss: 535.7420, Accuracy: 88.63%


100%|██████████| 1563/1563 [03:34<00:00,  7.29it/s]


Epoch 2, Loss: 202.0993, Accuracy: 95.83%


100%|██████████| 1563/1563 [03:34<00:00,  7.29it/s]

Epoch 3, Loss: 118.9406, Accuracy: 97.54%
